In [1]:
import sys
sys.path.insert(0,"/Users/mallen2/OpenPATH_Data/e-mission-server")

In [9]:
import emission.core.get_database as edb
import emission.storage.timeseries.abstract_timeseries as esta
#import emission.storage.timeseries.aggregate_timeseries as estag
#agts = estag.AggregateTimeSeries() 

In [10]:
programs_all = {"ens": [], "before_after": [], "only_after": []}  
# This will be a dict where keys are all pilot programs plus 
# "ens" for "ensemble, no stage" and "stage" for stage; 
# and values are the users corresponding to each
# Late in this process, I added "before_after" and "only_after": "only_after" is an ensemble for the programs 
# where users were given the app with the update already installed, 
# and "before_afer" is an ensemble for the remaining programs
only_after_programs = ["vail", "pc"]
for u in edb.get_uuid_db().find():         # add users to proper locations in programs 
    program = u["user_email"].split("_")[0]    # This info is in the Stage_uuids collection of the database
    uuid = u["uuid"]
    u["program"] = program
    if program not in programs_all.keys(): programs_all[program] = []
    if program != "stage":
        programs_all["ens"].append(uuid)
        if program in only_after_programs:
            programs_all["only_after"].append(uuid)
        else:
            programs_all["before_after"].append(uuid)
    programs_all[u["program"]].append(uuid)

print("Programs all: "+str({k: len(programs_all[k]) for k in programs_all}))

# Ignore the small ensembles in certain cases
programs_some = programs_all.copy()
programs_some.pop("before_after")
programs_some.pop("only_after")
print("Programs some: "+str({k: len(programs_some[k]) for k in programs_some}))

programs = programs_all  # For backwards compatibility
# The upside to this way of doing ensembles is it's really easy. The downside is a given user's data is calculated as many times as that user appears in the list of programs -- so with our current ensembles, we're doing most calculations three times when we only really need to do them once.

Programs all: {'ens': 163, 'before_after': 116, 'only_after': 47, 'vail': 6, 'sc': 16, '4c': 16, 'fc': 32, 'pc': 41, 'stage': 82, 'cc': 52}
Programs some: {'ens': 163, 'vail': 6, 'sc': 16, '4c': 16, 'fc': 32, 'pc': 41, 'stage': 82, 'cc': 52}


In [4]:
len([doc for doc in edb.get_uuid_db().find()])

245

In [13]:
confirmed_trip_df_map = {}
for u in edb.get_uuid_db().find():
    uuid = u['uuid']
    ts = esta.TimeSeries.get_time_series(uuid)
    ct_df = ts.get_data_df("analysis/confirmed_trip") 
    confirmed_trip_df_map[uuid] = ct_df

In [ ]:
'''labeling_fractions = {}
for u in confirmed_trip_df_map:
    ct_df = confirmed_trip_df_map[u]
    n_user_labels = sum(ct_df['user_input'] != {}) #sum(ct_df['algorithm_chosen'] == 'user_input')

    labeling_fractions[u] = user_labels/len(ct_df)

    print(f"Total trips: {len(ct_df)}. Number of labels: {user_labels}. Number inferred: {len(ct_df) - user_labels}")
    print(f"Labeling fraction: {user_labels/len(ct_df)}")'''

In [24]:
format_frac_percent = lambda num, denom: f"{num}/{denom}={(num/denom if denom != 0 else float('NaN')):.2%}"

la_user_training_prop = {}
program_labeling_prop = {}
good_la_model_prop = {}
strong_la_model_prop = {}
for program in programs_some:
    # find the proportion of users who have over ten labeled trips.
    #if program == 'ens': continue
    trip_sum = 0
    labeled_sum = 0
    enough_labeled_count = 0
    strong_model_count = 0
    good_model_count = 0
    for u in programs_some[program]:
        ct_df = confirmed_trip_df_map[u]
        if len(ct_df) == 0: continue
        n_user_labels = sum(ct_df['user_input'] != {})

        trip_sum += len(ct_df)
        labeled_sum += n_user_labels
        #n_u_l = sum(ct_df['algorithm_chosen'] == 'user_input')

        #assert n_user_labels == n_u_l

        if n_user_labels > 10:
            enough_labeled_count +=1
            if n_user_labels/len(ct_df) >= 0.8:
                strong_model_count += 1  # meaning this user has a high proportion of labeled trips
        if n_user_labels > 200:
            good_model_count += 1
    
    strong_la_model_prop[program] = format_frac_percent(strong_model_count,len(programs_some[program]))
    la_user_training_prop[program] = format_frac_percent(enough_labeled_count,len(programs_some[program]))
    program_labeling_prop[program] = format_frac_percent(labeled_sum,trip_sum)

    good_la_model_prop[program] = format_frac_percent(good_model_count,len(programs_some[program]))

print(f"Proportion of users with at least 10 labeled trips: {la_user_training_prop}\n")
print(f"Trip labeling proportion overall: {program_labeling_prop}\n")
print(f"Users with over 200 labeled trips: {good_la_model_prop}\n")
print(f"Users with at least 80% of trips labeled: {strong_la_model_prop}\n")

            

Proportion of users with at least 10 labeled trips: {'ens': '132/163=80.98%', 'vail': '6/6=100.00%', 'sc': '12/16=75.00%', '4c': '14/16=87.50%', 'fc': '22/32=68.75%', 'pc': '35/41=85.37%', 'stage': '39/82=47.56%', 'cc': '43/52=82.69%'}

Trip labeling proportion overall: {'ens': '50395/111650=45.14%', 'vail': '2040/2821=72.31%', 'sc': '5318/10805=49.22%', '4c': '4382/10121=43.30%', 'fc': '7856/19954=39.37%', 'pc': '11253/27337=41.16%', 'stage': '7334/23821=30.79%', 'cc': '19546/40612=48.13%'}

Users with over 200 labeled trips: {'ens': '76/163=46.63%', 'vail': '2/6=33.33%', 'sc': '9/16=56.25%', '4c': '7/16=43.75%', 'fc': '14/32=43.75%', 'pc': '16/41=39.02%', 'stage': '11/82=13.41%', 'cc': '28/52=53.85%'}

Users with at least 80% of trips labeled: {'ens': '38/163=23.31%', 'vail': '2/6=33.33%', 'sc': '3/16=18.75%', '4c': '2/16=12.50%', 'fc': '8/32=25.00%', 'pc': '11/41=26.83%', 'stage': '8/82=9.76%', 'cc': '12/52=23.08%'}



In [ ]:
print(ct_df)

In [ ]:
len(programs_some['ens'])

In [ ]:
for ct_df in confirmed_trip_df_map.values():
    print(ct_df.algorithm_chosen.value_counts())

In [ ]:
sum(ct_df['algorithm_chosen'] == 'user_input')